In [118]:
import pandas as pd
import numpy as np

import pandascharm as pc
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")

In [370]:
df = pd.read_excel('Review Paper Data Final.xlsx')

In [371]:
# Download
# pip install pandas-charm

In [372]:
sequence=df["Sequence"]
organisms=df["Organism Name"]

count=0
ofile = open("seqs.fasta", "w")
for i in range(len(sequence)):
    if str(sequence[i]) != 'nan':
        ofile.write(">" + str(organisms[i]) + "\n" + str(sequence[i]) + "\n")
        count=count+1
ofile.close()

In [373]:
from Bio.Align.Applications import ClustalwCommandline
in_file="seqs.fasta"
clustalw_cline=ClustalwCommandline("clustalw2", infile=in_file)

print(clustalw_cline)

clustalw2 -infile=seqs.fasta


In [374]:
# import os
# clustalw_exe =  r"/Users/ryanjang123/Documents/clustalw/clustalw2.exe"
# clustalw_cline=ClustalwCommandline(clustalw_exe, infile='seqs.fasta')

In [375]:
# assert os.path.isfile(clustalw_exe), 'Clustal W executable is missing'
# stdout = clustalw_cline()
# print(clustalw_cline)

check = clustalw_cline.align
print(check)

False


In [376]:
from Bio import AlignIO
ClustalAlign = AlignIO.read('seqs.aln', 'clustal')
summary_align = AlignInfo.SummaryInfo(ClustalAlign )

In [377]:
dframe = pc.from_bioalignment(ClustalAlign)

In [378]:
dframe

,AgrobacteriumTumefaciens5A,AgrobacteriumTumefaciens5AW127,AgrobacteriumTumefaciens5AL178,AgrobacteriumTumefaciens5AL178,AgrobacteriumTumefaciens5AC174,AgrobacteriumTumefaciens5AH229,AgrobacteriumTumefaciens5AV176,UnculturedBacteriumBgl1B,UnculturedBacteriumBgl1BH184F,UnculturedBacteriumBgl1BL409E,...,ThermofilumPendens,CellulomonasFimiATCC484D,CellulomonasFimiATCC484C,ClavisporaSpNRRLY50464BGL1,WeissellaCibaria,CellulomonasFimiATCC484B,CellulomonasFimiNag3A,ThermobifidaFuscaYX,CellulomonasFimiHex20A,VibrioCholerae
0,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,-,-,-,-,-,-,-,-,-,-,...,-,E,-,-,-,-,-,-,-,-
1293,-,-,-,-,-,-,-,-,-,-,...,-,K,-,-,-,-,-,-,-,-
1294,-,-,-,-,-,-,-,-,-,-,...,-,A,-,-,-,-,-,-,-,-
1295,-,-,-,-,-,-,-,-,-,-,...,-,R,-,-,-,-,-,-,-,-


In [278]:
dframe['AgrobacteriumTumefaciens5A']

0       -
1       -
2       -
3       -
4       -
       ..
1292    -
1293    -
1294    -
1295    -
1296    -
Name: AgrobacteriumTumefaciens5A, Length: 1297, dtype: category
Categories (21, object): ['-', 'A', 'C', 'D', ..., 'T', 'V', 'W', 'Y']

In [416]:
# changing U or O to -
dframe = dframe.replace('U', '-')
dframe = dframe.replace('O','-')

In [323]:
data=df[['Wild/Mutant', 'Kingdom', 'pH Optimum', 'Temperature Optimum',
       't1/2 (min)', 'kd (min-1)', 'pNP-Glc Km (mM)', 'pNP-Glc kcat (1/s)',
       'pNP-Glc kcat/Km (1/smM)', 'Cellobiose Km (mM)',
       'Cellobiose kcat (1/s)', 'Cellobiose kcat/Km (1/smM)', 'Cellobiose Ki (mM)',
       'pNP-Glc Ki (mM)', 'MW (kDa)']]

In [324]:
general_info=data[['Wild/Mutant','Kingdom','pH Optimum','Temperature Optimum','MW (kDa)']]
stability_df=data[['t1/2 (min)','kd (min-1)']]
kinetics_pnp=data[['pNP-Glc Km (mM)', 'pNP-Glc kcat (1/s)', 'pNP-Glc kcat/Km (1/smM)', 'pNP-Glc Ki (mM)']]
kinetics_cellobiose=data[['Kingdom','Cellobiose Km (mM)', 'Cellobiose kcat (1/s)', 'Cellobiose kcat/Km (1/smM)', 'Cellobiose Ki (mM)']]


#removal of outliers

def removeoutliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out.dropna()

In [325]:
kcatPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
kcatPnpGlc

,pNP-Glc kcat (1/s)
0,26.640000
2,5.500000
3,1.700000
4,169.000000
5,277.000000
...,...
309,19.848533
310,14.464000
311,35.558400
312,20.250000


In [326]:
kcatPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
KmPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc Km (mM)']])
kcatKmPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc kcat/Km (1/smM)']])
KiPnpGlc=removeoutliers(kinetics_pnp[['pNP-Glc Ki (mM)']])
stability=removeoutliers(stability_df[['t1/2 (min)']])
# need a new one k/k

kcat_index = list(kcatPnpGlc.index)
Km_index = list(KmPnpGlc.index)
kcatKm_index = list(kcatKmPnpGlc.index)
Ki_index = list(KiPnpGlc.index)
stability_index = list(stability.index)

# kcatKmCellobiose=kinetics_cellobiose[['Cellobiose kcat/Km (1/smM)']]
# kcatCellobiose=kinetics_cellobiose[[ 'Cellobiose kcat (1/s)']]
# KmCellobiose=kinetics_cellobiose[['Cellobiose Km (mM)']]

In [327]:
# Machine Learning Tools

def LR(X_train, y_train, X_test):
    from sklearn.linear_model import LinearRegression
    lm = LinearRegression()
    lm.fit(X_train,y_train)
    lin_pred = lm.predict(X_test)
    pred_training= lm.predict(X_train)
    
    return lin_pred , pred_training

#Ordinary Least Squares Regression
def OLS(X_train, y_train, X_test):
    import statsmodels.api as sm
    from scipy import stats
    X2 = sm.add_constant(X_train)
    est = sm.OLS(y_train,X2)
    est2 = est.fit()
    print(est2.summary())
    coefficients=est2.params[1:]
    intercept=est2.params[0]
    OLS_pred=(coefficients*X_test).sum(axis = 1, skipna = True) + intercept
    
    return OLS_pred

#Lasso regression

def LASSO(X_train, y_train, X_test):
    param_grid={'alpha':np.arange(0,1.1,0.05).tolist()}
    lasso_reg=Lasso()
    grid_search=GridSearchCV(lasso_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    lasso_pred=grid_search.predict(X_test)
    pred_training= grid_search.predict(X_train)
    return lasso_pred, pred_training

#Partial Least Square Regression

def PLS(X_train, y_train, X_test):
    param_grid={'n_components':np.arange(1,10,1).tolist()}
    pls = PLSRegression()
    grid_search=GridSearchCV(pls, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    pls.fit(scale(X_train), y_train)
    PLS_pred=pls.predict(scale(X_test))
    pred_training= pls.predict(X_train)
    
    return PLS_pred , pred_training

#Random Forest

def RF(X_train, y_train, X_test):
    param_grid={"n_estimators": [100] , "max_features": [5], 'random_state':[20]}
    rf = RandomForestRegressor()
    grid_search=GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    RF_pred = grid_search.predict(X_test)
    pred_training= grid_search.predict(X_train)
    return RF_pred, pred_training

#Decision Tree

def DT(X_train, y_train, X_test):
    tree_reg=DecisionTreeRegressor()
    tree_reg.fit(X_train, y_train)
    DT_pred=tree_reg.predict(X_test)
    pred_training= tree_reg.predict(X_train)
    return DT_pred, pred_training

#SVM

def SVM(X_train, y_train, X_test):
    model = SVR(kernel='poly', degree=2, C=100, epsilon=0.5)
    model.fit(X_train,y_train)
    SVM_pred = model.predict(X_test)
    pred_training= model.predict(X_train)
    return SVM_pred, pred_training

#Neural Network Regression

def NNR(X_train, y_train, X_test):
    regr = MLPRegressor(random_state=101, max_iter=100).fit(X_train, y_train)
    NNR_pred=regr.predict(X_test)
    pred_training= regr.predict(X_train)
    return NNR_pred, pred_training

#Elastic Net

def EN(X_train, y_train, X_test):
    ENmodel = ElasticNet(alpha=1.0, l1_ratio=0.5)
    ENmodel.fit(X_train,y_train)
    EN_pred = ENmodel.predict(X_test)
    pred_training= ENmodel.predict(X_train)
    return EN_pred, pred_training

In [328]:
# 1 pNp OneHot Encoder
X = df['Sequence'].iloc[kcat_index].dropna()
rem_kcat_index = list(X.index)
y = kinetics_pnp['pNP-Glc kcat (1/s)'].iloc[rem_kcat_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [329]:
encoded.transform(temp_seq).toarray().shape

(152, 145)

In [330]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [331]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [332]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [333]:
results

[['Linear Regression',
  20.87136713585657,
  1563786471100599.0,
  0.9383336465181893,
  -8.815443261662365e+26],
 ['LASSO Regression',
  58.900993075962226,
  55.783322885591524,
  0.5088753785207363,
  -0.12175499663922529],
 ['Partial Least Square',
  76.71268037983447,
  60.25392772671232,
  0.16693135681609184,
  -0.3087598877125477],
 ['Decision Tree Regression',
  20.543864385424413,
  56.02933975553024,
  0.9402537362187964,
  -0.13167119326597843],
 ['Random Forest Regression',
  37.64781593341234,
  53.53864320196452,
  0.799356060678127,
  -0.033294156260743835],
 ['Support Vector Machine Regression',
  48.945761410890114,
  54.34909080402157,
  0.6608618469500656,
  -0.06481416031116027],
 ['Neural Network Regression',
  95.40190747973072,
  55.335339925253066,
  -0.28842929020594066,
  -0.1038102325505239],
 ['Elastic Network Regression',
  83.25897417302092,
  60.12521429800463,
  0.018684668032005525,
  -0.3031743581542454]]

In [334]:
# 2 KmpNp OneHot Encoder
X = df['Sequence'].iloc[Km_index].dropna()
rem_km_index = list(X.index)
y = kinetics_pnp['pNP-Glc Km (mM)'].iloc[rem_km_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [335]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [336]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [337]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [338]:
results

[['Linear Regression',
  0.04939499633896624,
  5214448561005.287,
  0.9997732539192618,
  -3.2625723303733013e+24],
 ['LASSO Regression',
  0.04875596534643474,
  2.992652231868433,
  0.9997790828695939,
  -0.07462236916404064],
 ['Partial Least Square',
  2.984248008324353,
  2.804343321023644,
  0.17235575541514714,
  0.056361318152093554],
 ['Decision Tree Regression',
  0.04875596534643475,
  3.8401179868705837,
  0.9997790828695939,
  -0.7694264287860066],
 ['Random Forest Regression',
  1.1614918891922572,
  2.7954567984676717,
  0.8746263594841535,
  0.062332327435937196],
 ['Support Vector Machine Regression',
  0.4792546596421556,
  2.8125274145921306,
  0.9786544887020214,
  0.05084551969886297],
 ['Neural Network Regression',
  3.2114977842891914,
  3.302782797293149,
  0.04150659862794537,
  -0.308890757917768],
 ['Elastic Network Regression',
  3.2802962164416862,
  2.89711563269851,
  0.0,
  -0.007105650215459169]]

In [339]:
# 3 kcat/Km OneHot Encoder
X = df['Sequence'].iloc[kcatKm_index].dropna()
rem_index = list(X.index)
y = kinetics_pnp['pNP-Glc kcat/Km (1/smM)'].iloc[rem_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [340]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [341]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [342]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [343]:
results

[['Linear Regression',
  1.0253847322850078,
  101.36102570695247,
  0.9999663848376071,
  0.25046939195839657],
 ['LASSO Regression',
  1.0253872763220617,
  98.8760057667968,
  0.9999663846708047,
  0.2867706482675253],
 ['Partial Least Square',
  160.9564018476239,
  112.6071783556726,
  0.17171836860144485,
  0.07491948860852149],
 ['Decision Tree Regression',
  1.0253847322850078,
  100.37074647524769,
  0.9999663848376071,
  0.2650434117167906],
 ['Random Forest Regression',
  65.2747781617935,
  101.19244125811966,
  0.8637764074589283,
  0.25296056887283813],
 ['Support Vector Machine Regression',
  146.1189141611958,
  107.89780497544254,
  0.31738725474279716,
  0.1506775604368411],
 ['Neural Network Regression',
  193.92080799695012,
  122.64631093651391,
  -0.20229307175088374,
  -0.09737827583723213],
 ['Elastic Network Regression',
  174.63551584605014,
  124.46997448081302,
  0.02495050176340663,
  -0.13025537282360888]]

In [344]:
# 4 Ki
X = df['Sequence'].iloc[Ki_index].dropna()
rem_index = list(X.index)
y = kinetics_pnp['pNP-Glc Ki (mM)'].iloc[rem_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [345]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [346]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [347]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [348]:
results

[['Linear Regression',
  3.10955550706039e-13,
  364.30522096519564,
  1.0,
  -0.11658295556824227],
 ['LASSO Regression',
  0.004869183143075672,
  367.80779891094505,
  0.9999999997401353,
  -0.13815673045515253],
 ['Partial Least Square',
  252.4145866458684,
  361.47810677074364,
  0.30166492622158325,
  -0.09932018326104308],
 ['Decision Tree Regression', 0.0, 514.5322750518183, 1.0, -1.227334517239759],
 ['Random Forest Regression',
  114.23170386185485,
  401.00702824518294,
  0.8569762148992024,
  -0.3528952508484202],
 ['Support Vector Machine Regression',
  262.09779890268965,
  415.91206746497966,
  0.24705768411180584,
  -0.4553359124457106],
 ['Neural Network Regression',
  392.7496275168205,
  515.5164017556967,
  -0.690699879186639,
  -1.2358629448696488],
 ['Elastic Network Regression',
  286.86656703340213,
  368.61913121176576,
  0.09802431471585704,
  -0.14318349602439095]]

In [349]:
# 5 stability
X = df['Sequence'].iloc[stability_index].dropna()
rem_index = list(X.index)
y = stability_df['t1/2 (min)'].iloc[rem_index]
one_hot = OneHotEncoder()
temp_seq = np.array(X).reshape(-1,1)
encoded = one_hot.fit(temp_seq)
X = encoded.transform(temp_seq).toarray()

In [350]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [351]:
lin_pred, lin_train = LR(X_train, y_train, X_test)
lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
DT_pred , DT_train = DT(X_train, y_train, X_test)
RF_pred , RF_train = RF(X_train, y_train, X_test)
SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
EN_pred , EN_train = EN(X_train, y_train, X_test)

In [352]:
results=[]
results.append(["Linear Regression",  np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
results.append(["LASSO Regression",   np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
results.append(["Partial Least Square",np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
results.append(["Decision Tree Regression",  np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
results.append(["Random Forest Regression",np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
results.append(["Support Vector Machine Regression", np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
results.append(["Neural Network Regression", np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
results.append(["Elastic Network Regression", np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 

In [353]:
results

[['Linear Regression',
  103.86535755486524,
  262.6332045603385,
  0.8334903943861203,
  -0.07231225572187161],
 ['LASSO Regression',
  103.86535755486524,
  258.75083355256436,
  0.8334903943861203,
  -0.04084370293356532],
 ['Partial Least Square',
  220.902201406519,
  257.90201331901267,
  0.2468214573103512,
  -0.034026023855763254],
 ['Decision Tree Regression',
  103.86535755486524,
  307.8598226778517,
  0.8334903943861203,
  -0.47342499813741856],
 ['Random Forest Regression',
  129.41587210598405,
  274.4121380542155,
  0.7414926249406171,
  -0.17065423126060963],
 ['Support Vector Machine Regression',
  230.37378639346943,
  288.79351099160556,
  0.18084898839738028,
  -0.29657267711598556],
 ['Neural Network Regression',
  309.5415034962932,
  338.4621633726449,
  -0.4788891631650636,
  -0.7809111628267713],
 ['Elastic Network Regression',
  244.39472616487058,
  258.1714756340616,
  0.07810489057894177,
  -0.03618790388824955]]

In [354]:
# Outliers comparison dataframe

AA1=kinetics_pnp['pNP-Glc kcat (1/s)'].dropna()
AA1_ind = list(AA1.index)
A1 = df['Sequence'].iloc[AA1_ind].dropna()


AA2=kinetics_pnp['pNP-Glc Km (mM)'].dropna()
AA2_ind = list(AA2.index)
A2 = df['Sequence'].iloc[AA2_ind].dropna()

AA3=kinetics_pnp['pNP-Glc kcat/Km (1/smM)'].dropna()
AA3_ind = list(AA3.index)
A3 = df['Sequence'].iloc[AA3_ind].dropna()


AA4=kinetics_pnp['pNP-Glc Ki (mM)'].dropna()
AA4_ind = list(AA4.index)
A4 = df['Sequence'].iloc[AA4_ind].dropna()

AA5=stability_df['t1/2 (min)'].dropna()
AA5_ind = list(AA5.index)
A5 = df['Sequence'].iloc[AA5_ind].dropna()

BB1 = removeoutliers(kinetics_pnp[['pNP-Glc kcat (1/s)']])
BB1_ind = list(BB1.index)
B1 = df['Sequence'].iloc[BB1_ind].dropna()

BB2 = removeoutliers(kinetics_pnp[['pNP-Glc Km (mM)']])
BB2_ind = list(BB2.index)
B2 = df['Sequence'].iloc[BB2_ind].dropna()

BB3 = removeoutliers(kinetics_pnp[['pNP-Glc kcat/Km (1/smM)']])
BB3_ind = list(BB3.index)
B3 = df['Sequence'].iloc[BB3_ind].dropna()

BB4= removeoutliers(kinetics_pnp[['pNP-Glc Ki (mM)']])
BB4_ind = list(BB4.index)
B4 = df['Sequence'].iloc[BB4_ind].dropna()

BB5 = removeoutliers(stability_df[['t1/2 (min)']])
BB5_ind = list(BB5.index)
B5 = df['Sequence'].iloc[BB5_ind].dropna()

In [355]:
frame = pd.DataFrame( columns=['w/outliers','w/o outliers' ])
dict1 = pd.Series({'w/outliers':len(A1) ,'w/o outliers':len(B1)}, name = 'pNP-Glc kcat (1/s)')
dict2 = pd.Series({'w/outliers':len(A2),'w/o outliers':len(B2)}, name = 'pNP-Glc Km (mM)')
dict3 = pd.Series({'w/outliers':len(A3),'w/o outliers':len(B3)},name = 'pNP-Glc kcat/Km (1/smM)')
dict4 = pd.Series({'w/outliers':len(A4),'w/o outliers':len(B4)}, name = 'pNP-Glc Ki (mM)')
dict5 = pd.Series({'w/outliers':len(A5),'w/o outliers':len(B5)}, name = 't1/2 (min)')

In [356]:
frame = frame.append(dict1)
frame = frame.append(dict2)
frame = frame.append(dict3)
frame =frame.append(dict4)
frame = frame.append(dict5)

In [357]:
frame

,w/outliers,w/o outliers
pNP-Glc kcat (1/s),176,152
pNP-Glc Km (mM),191,171
pNP-Glc kcat/Km (1/smM),177,154
pNP-Glc Ki (mM),58,46
t1/2 (min),62,50


In [379]:
#AAIndex
from pyaaisc import Aaindex
aaindex = Aaindex()
full_list = aaindex.get_all(dbkey='aaindex1')

In [417]:
record = aaindex.get(full_list[0][0])
record.title

'Peptide/protein structure analysis using the chemical shift index method: upfield alpha-CH values reveal dynamic helices and aL sites'

In [418]:
c = record.index_data

In [419]:
c['-'] = 0

In [429]:
sequence_aligned=dframe.apply(lambda dframe : pd.Series(c[i] for i in dframe))

In [426]:
# org=list(dframe.columns)
# sequence_aligned.set_axis(org, axis=1, inplace=True)

In [433]:
sequence_aligned['AgrobacteriumTumefaciens5A']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1292    0.0
1293    0.0
1294    0.0
1295    0.0
1296    0.0
Name: AgrobacteriumTumefaciens5A, Length: 1297, dtype: float64

In [435]:
X = sequence_aligned.transpose()

In [437]:
X.iloc[0]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1292    0.0
1293    0.0
1294    0.0
1295    0.0
1296    0.0
Name: AgrobacteriumTumefaciens5A, Length: 1297, dtype: float64

In [ ]:
for key in range(len(full_list)):
    try:
        record = aaindex.get(full_list[key][0])
        title = record.title
        index_data = record.index_data
        index_data['-'] = 0
        sequence_aligned=dframe.apply(lambda dframe : pd.Series(index_data[i] for i in dframe))

        org=list(dframe.index)
        sequence_aligned.set_axis(org, axis=0, inplace=True)
        X=sequence_aligned
        Z=pd.concat([X, y], axis=1).dropna()

        X=Z.loc[:, Z.columns != 'pNP-Glc kcat (1/s)']
        y=Z.loc[:, Z.columns == 'pNP-Glc kcat (1/s)']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

        #turn y dataframe to array for easier calculations
        y_test = y_test.to_numpy()
        
        lin_pred, lin_train = LR(X_train, y_train, X_test)
        lasso_pred , lasso_train = LASSO(X_train, y_train, X_test)
        PLS_pred , PLS_train = PLS(X_train, y_train, X_test)
        DT_pred , DT_train = DT(X_train, y_train, X_test)
        RF_pred , RF_train = RF(X_train, y_train, X_test)
        SVM_pred , SVM_train = SVM(X_train, y_train, X_test)
        NNR_pred, NNR_train = NNR(X_train, y_train, X_test)
        EN_pred , EN_train = EN(X_train, y_train, X_test)


        # Add rows 
        results.append(["Linear Regression", full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, lin_train)), np.sqrt(metrics.mean_squared_error(y_test, lin_pred)), r2_score(y_train, lin_train),r2_score(y_test, lin_pred)]) 
        results.append(["LASSO Regression",  full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, lasso_train)), np.sqrt(metrics.mean_squared_error(y_test, lasso_pred)), r2_score(y_train, lasso_train),r2_score(y_test, lasso_pred)]) 
        results.append(["Partial Least Square", full_list[key][0], title,np.sqrt(metrics.mean_squared_error(y_train, PLS_train)), np.sqrt(metrics.mean_squared_error(y_test, PLS_pred)), r2_score(y_train, PLS_train),r2_score(y_test, PLS_pred)]) 
        results.append(["Decision Tree Regression", full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_test, DT_pred)), r2_score(y_train, DT_train),r2_score(y_test, DT_pred)]) 
        results.append(["Random Forest Regression",full_list[key][0],  title, np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_test, RF_pred)), r2_score(y_train, RF_train),r2_score(y_test, RF_pred)]) 
        results.append(["Support Vector Machine Regression",full_list[key][0],  title, np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_test, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_test, SVM_pred)]) 
        results.append(["Neural Network Regression", full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_test, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_test, NNR_pred)]) 
        results.append(["Elastic Network Regression",full_list[key][0], title, np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_test, EN_pred)), r2_score(y_train, EN_train),r2_score(y_test, EN_pred)]) 
    except:
            pass

In [358]:
#BLOSUM 62
blosum.update(((b,a),val) for (a,b),val in list(blosum.items()))

def score_pairwise(seq1, seq2, matrix, gap_s, gap_e, gap = True):
    for A,B in zip(seq1, seq2):
        diag = ('-'==A) or ('-'==B)
        yield (gap_e if gap else gap_s) if diag else matrix[(A,B)]
        gap = diag

In [359]:
a = dframe['AgrobacteriumTumefaciens5A']
b = dframe['AgrobacteriumTumefaciens5AW127']

In [360]:
print(sum(score_pairwise(a, b, blosum, -5, -1)))

1521


In [363]:
n

159

In [361]:
n = len(dframe.columns)
enc_seq = np.zeros((n,n))

In [364]:
i = 0

for a in dframe.columns:
    j = 0
    for b in dframe.columns:
        enc_seq[i,j] = sum(score_pairwise(dframe[a], dframe[b], blosum, -5, -1))
        j += 1
    i += 1

KeyError: ('AgrobacteriumTumefaciens5AL178', 'AgrobacteriumTumefaciens5AL178')

In [365]:
dframe[a]

,AgrobacteriumTumefaciens5AL178,AgrobacteriumTumefaciens5AL178
0,-,-
1,-,-
2,-,-
3,-,-
4,-,-
...,...,...
1292,-,-
1293,-,-
1294,-,-
1295,-,-


In [263]:
set(list(dframe[a]))

{'-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y'}

In [366]:
dframe[b]

,AgrobacteriumTumefaciens5AL178,AgrobacteriumTumefaciens5AL178
0,-,-
1,-,-
2,-,-
3,-,-
4,-,-
...,...,...
1292,-,-
1293,-,-
1294,-,-
1295,-,-


In [264]:
set(list(dframe[b]))

{'-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y'}